In [1]:
from math import log, exp, sqrt
from scipy.integrate import quad
import numpy as np

def test_py1(x, a, b):
    return exp(log(x) * a) + b

def test_py2(x, c):
    return sqrt(x) / c

def integrand_py(x, a, b, c):
    return test_py1(x, a, b) * test_py2(x, c)

def integ_py(vec, *args):
    n = vec.shape[0]
    res = np.empty(n-1)
    for i in range(n-1):
        res[i] = quad(integrand_py, vec[i], vec[i+1], args=(*args,))[0]
    return res

In [11]:
DTYPE = np.double

vec = np.random.random(100).astype(DTYPE)
a = 2.0
b = 0.5
c = 1.5

In [56]:
%timeit integrand_py(10.0, a, b, c)

438 ns ± 2.74 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [12]:
%timeit integ_py(vec, a, b, c)

1.3 ms ± 49 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Simply convert to Cython

In [13]:
%load_ext cython

In [16]:
%%cython
from math import log, exp, sqrt
from scipy.integrate import quad
import numpy as np

def test_cy1(x, a, b):
    return exp(log(x) * a) + b

def test_cy2(x, c):
    return sqrt(x) / c

def integrand_cy(x, a, b, c):
    return test_cy1(x, a, b) * test_cy2(x, c)

def integ_cy(vec, *args):
    n = vec.shape[0]
    res = np.empty(n-1)
    for i in range(n-1):
        res[i] = quad(integrand_cy, vec[i], vec[i+1], args=(*args,))[0]
    return res

In [17]:
%timeit integ_cy(vec, a, b, c)

963 µs ± 17.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Convert to C math library

In [18]:
%%cython
from libc.math cimport log, exp, sqrt
from scipy.integrate import quad
import numpy as np

def test_cy1(x, a, b):
    return exp(log(x) * a) + b

def test_cy2(x, c):
    return sqrt(x) / c

def integrand_cy(x, a, b, c):
    return test_cy1(x, a, b) * test_cy2(x, c)

def integ_cy(vec, *args):
    n = vec.shape[0]
    res = np.empty(n-1)
    for i in range(n-1):
        res[i] = quad(integrand_cy, vec[i], vec[i+1], args=(*args,))[0]
    return res

In [19]:
%timeit integ_cy(vec, a, b, c)

740 µs ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Add static types to test functions

In [20]:
%%cython
from libc.math cimport log, exp, sqrt
from scipy.integrate import quad
import numpy as np

def test_cy1(double x, double a, double b):
    return exp(log(x) * a) + b

def test_cy2(double x, double c):
    return sqrt(x) / c

def integrand_cy(x, a, b, c):
    return test_cy1(x, a, b) * test_cy2(x, c)

def integ_cy(vec, *args):
    n = vec.shape[0]
    res = np.empty(n-1)
    for i in range(n-1):
        res[i] = quad(integrand_cy, vec[i], vec[i+1], args=(*args,))[0]
    return res

In [21]:
%timeit integ_cy(vec, a, b, c)

649 µs ± 2.39 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [22]:
%%cython
from libc.math cimport log, exp, sqrt
from scipy.integrate import quad
import numpy as np

def test_cy1(double x, double a, double b):
    return exp(log(x) * a) + b

def test_cy2(double x, double c):
    return sqrt(x) / c

# x to double x
def integrand_cy(double x, double a, double b, double c):
    return test_cy1(x, a, b) * test_cy2(x, c)

def integ_cy(vec, *args):
    n = vec.shape[0]
    res = np.empty(n-1)
    for i in range(n-1):
        res[i] = quad(integrand_cy, vec[i], vec[i+1], args=(*args,))[0]
    return res

In [23]:
%timeit integ_cy(vec, a, b, c)

719 µs ± 6.59 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Why is it slower????

In [24]:
%%cython
from libc.math cimport log, exp, sqrt
from scipy.integrate import quad
import numpy as np

cdef double test_cy1(double x, double a, double b):
    return exp(log(x) * a) + b

cdef double test_cy2(double x, double c):
    return sqrt(x) / c

def integrand_cy(x, a, b, c):
    return test_cy1(x, a, b) * test_cy2(x, c)

def integ_cy(vec, *args):
    n = vec.shape[0]
    res = np.empty(n-1)
    for i in range(n-1):
        res[i] = quad(integrand_cy, vec[i], vec[i+1], args=(*args,))[0]
    return res

In [25]:
%timeit integ_cy(vec, a, b, c)

492 µs ± 11.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Disable zero division check

In [57]:
%%cython
from libc.math cimport log, exp, sqrt
from scipy.integrate import quad
import numpy as np
import cython

cdef double test_cy1(double x, double a, double b):
    return exp(log(x) * a) + b

@cython.cdivision(True)
cdef double test_cy2(double x, double c):
    return sqrt(x) / c

def integrand_cy(x, a, b, c):
    return test_cy1(x, a, b) * test_cy2(x, c)

def integ_cy(vec, *args):
    n = vec.shape[0]
    res = np.empty(n-1)
    for i in range(n-1):
        res[i] = quad(integrand_cy, vec[i], vec[i+1], args=(*args,))[0]
    return res

In [59]:
%timeit integrand_cy(10.0, a, b, c)

138 ns ± 1.39 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [34]:
%timeit integ_cy(vec, a, b, c)

484 µs ± 14.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Add static types to integrand

In [41]:
%%cython
from libc.math cimport log, exp, sqrt
from scipy.integrate import quad
import numpy as np
import cython

cdef double test_cy1(double x, double a, double b):
    return exp(log(x) * a) + b

@cython.cdivision(True)
cdef double test_cy2(double x, double c):
    return sqrt(x) / c

def integrand_cy(double x, double a, double b, double c):
    return test_cy1(x, a, b) * test_cy2(x, c)

def integ_cy(vec, *args):
    n = vec.shape[0]
    res = np.empty(n-1)
    for i in range(n-1):
        res[i] = quad(integrand_cy, vec[i], vec[i+1], args=(*args,))[0]
    return res

In [36]:
%timeit integ_cy(vec, a, b, c)

489 µs ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [42]:
%%cython
from libc.math cimport log, exp, sqrt
from scipy.integrate import quad
import numpy as np
import cython

cdef double test_cy1(double x, double a, double b):
    return exp(log(x) * a) + b

@cython.cdivision(True)
cdef double test_cy2(double x, double c):
    return sqrt(x) / c

cdef double integrand_cy(double x, double a, double b, double c):
    return test_cy1(x, a, b) * test_cy2(x, c)

def integ_cy(vec, *args):
    n = vec.shape[0]
    res = np.empty(n-1)
    for i in range(n-1):
        res[i] = quad(integrand_cy, vec[i], vec[i+1], args=(*args,))[0]
    return res

In [39]:
%timeit integ_cy(vec, a, b, c)

497 µs ± 12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## No performance gain in defining integrand in static typed function

# Add static types for for-loops

In [43]:
%%cython
from libc.math cimport log, exp, sqrt
from scipy.integrate import quad
import numpy as np
import cython

cdef double test_cy1(double x, double a, double b):
    return exp(log(x) * a) + b

@cython.cdivision(True)
cdef double test_cy2(double x, double c):
    return sqrt(x) / c

def integrand_cy(x, a, b, c):
    return test_cy1(x, a, b) * test_cy2(x, c)

def integ_cy(vec, *args):
    cdef Py_ssize_t n = vec.shape[0]
    res = np.empty(n-1)
    cdef Py_ssize_t i
    for i in range(n-1):
        res[i] = quad(integrand_cy, vec[i], vec[i+1], args=(*args,))[0]
    return res

In [44]:
%timeit integ_cy(vec, a, b, c)

473 µs ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Converting to memoryview 

In [48]:
%%cython
from libc.math cimport log, exp, sqrt
from scipy.integrate import quad
import numpy as np
import cython

cdef double test_cy1(double x, double a, double b):
    return exp(log(x) * a) + b

@cython.cdivision(True)
cdef double test_cy2(double x, double c):
    return sqrt(x) / c

def integrand_cy(x, a, b, c):
    return test_cy1(x, a, b) * test_cy2(x, c)

def integ_cy(vec, *args):
    cdef Py_ssize_t n = vec.shape[0]
    res = np.empty(n-1)
    cdef double[:] resv = res
    cdef Py_ssize_t i
    for i in range(n-1):
        resv[i] = quad(integrand_cy, vec[i], vec[i+1], args=(*args,))[0]
    return res

In [46]:
%timeit integ_cy(vec, a, b, c)

488 µs ± 6.24 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [51]:
%%cython
from libc.math cimport log, exp, sqrt
from scipy.integrate import quad
import numpy as np
import cython

cdef double test_cy1(double x, double a, double b):
    return exp(log(x) * a) + b

@cython.cdivision(True)
cdef double test_cy2(double x, double c):
    return sqrt(x) / c

cdef double integrand_cy(double x, double a, double b, double c):
    return test_cy1(x, a, b) * test_cy2(x, c)

def integ_cy(double[:] vec, *args):
    cdef Py_ssize_t n = vec.shape[0]
    res = np.empty(n-1)
    cdef double[:] resv = res
    cdef Py_ssize_t i
    for i in range(n-1):
        resv[i] = quad(integrand_cy, vec[i], vec[i+1], args=(*args,))[0]
    return res

In [52]:
%timeit integ_cy(vec, a, b, c)

495 µs ± 7.32 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [54]:
np.allclose(integ_py(vec, a, b, c), integ_cy(vec, a, b, c))

True

# Memoryview is not effective since the result of quad function is not C type double.

# Using Python function in integrand

In [60]:
from math import exp
from scipy.integrate import quad
from scipy.special import erfc
import numpy as np

def test_py1(x, a, b):
    return erfc(x) * a + b

def test_py2(x, c):
    return exp(x) / c

def integrand_py(x, a, b, c):
    return test_py1(x, a, b) * test_py2(x, c)

In [62]:
a = 2.0
b = 3.0
c = 4.0

In [63]:
%timeit quad(integrand_py,0, 100, args=(a, b, c))

218 µs ± 6.16 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Simply converting to Cython

In [64]:
%%cython
from math import exp
from scipy.integrate import quad
from scipy.special import erfc
import numpy as np

def test_cy1(x, a, b):
    return erfc(x) * a + b

def test_cy2(x, c):
    return exp(x) / c

def integrand_cy(x, a, b, c):
    return test_cy1(x, a, b) * test_cy2(x, c)

In [65]:
%timeit quad(integrand_cy,0, 100, args=(a, b, c))

187 µs ± 826 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Convert to C library

In [80]:
%%cython
from libc.math cimport exp, erfc
from scipy.integrate import quad
import numpy as np

def test_cy1(x, a, b):
    return erfc(x) * a + b

def test_cy2(x, c):
    return exp(x) / c

def integrand_cy(x, a, b, c):
    return test_cy1(x, a, b) * test_cy2(x, c)

In [81]:
%timeit quad(integrand_cy,0, 100, args=(a, b, c))

30.5 µs ± 294 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [82]:
integrand_cy(1, 2, 3, 4) - integrand_py(1, 2, 3, 4)

0.0

# Static type for function 1

In [83]:
%%cython
from libc.math cimport exp, erfc
from scipy.integrate import quad
import numpy as np

cdef double test_cy1(double x, double a, double b):
    return erfc(x) * a + b

def test_cy2(x, c):
    return exp(x) / c

def integrand_cy(x, a, b, c):
    return test_cy1(x, a, b) * test_cy2(x, c)

In [84]:
%timeit quad(integrand_cy,0, 100, args=(a, b, c))

23.7 µs ± 183 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Static type for function 2

In [85]:
%%cython
from libc.math cimport exp, erfc
from scipy.integrate import quad
import numpy as np

cdef double test_cy1(double x, double a, double b):
    return erfc(x) * a + b

cdef double test_cy2(double x, double c):
    return exp(x) / c

def integrand_cy(x, a, b, c):
    return test_cy1(x, a, b) * test_cy2(x, c)

In [86]:
%timeit quad(integrand_cy,0, 100, args=(a, b, c))

15.7 µs ± 72.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [74]:
%prun quad(integrand_cy, 0, 100, args=(a, b, c))

         9 function calls in 0.001 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.001    0.001 {built-in method scipy.integrate._quadpack._qagse}
        1    0.000    0.000    0.001    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.001    0.001 quadpack.py:44(quad)
        1    0.000    0.000    0.001    0.001 quadpack.py:435(_quad)
        1    0.000    0.000    0.001    0.001 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.min}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.max}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [75]:
%prun quad(integrand_py, 0, 100, args=(a, b, c))

         597 function calls in 0.001 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      147    0.000    0.000    0.000    0.000 <ipython-input-60-2b26337c2046>:6(test_py1)
      147    0.000    0.000    0.001    0.000 <ipython-input-60-2b26337c2046>:12(integrand_py)
        1    0.000    0.000    0.001    0.001 {built-in method scipy.integrate._quadpack._qagse}
      147    0.000    0.000    0.000    0.000 <ipython-input-60-2b26337c2046>:9(test_py2)
        1    0.000    0.000    0.001    0.001 {built-in method builtins.exec}
      147    0.000    0.000    0.000    0.000 {built-in method math.exp}
        1    0.000    0.000    0.001    0.001 quadpack.py:44(quad)
        1    0.000    0.000    0.001    0.001 quadpack.py:435(_quad)
        1    0.000    0.000    0.001    0.001 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.max}
        1    0.000    0.000    0.000    0.000 {bu

In [79]:
%%cython
from libc.math cimport erf, erfc
print(erfc(1))

0.15729920705028513
